# Install Libraries and imports

In [1]:
!pip install fastai -Uqq

     |████████████████████████████████| 189 kB 5.2 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 


In [2]:
from fastai.tabular.all import *
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import r2_score
from torch.utils.data import Dataset

# Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = Path('/content/drive/MyDrive/colab_notebooks/algovera/defi')

# Preparing the data

In [5]:
df = pd.read_csv(path/'compound.csv')
df["Date"] = pd.to_datetime(df["Timestamp"], unit='s', origin='unix')

tokens = df["Token"].unique()
tokens

array(['DAI', 'USDC', 'USDT', 'ETH'], dtype=object)

In [6]:
df = df.drop_duplicates(['Timestamp', 'Token'])
counts = pd.DataFrame(df['Timestamp'].value_counts()).reset_index()
counts.columns = ['Timestamp', 'Counts']
df = df.merge(counts, on='Timestamp')

In [7]:
df = df[df['Counts'] == 4].reset_index(drop=True).drop('Counts', axis=1)

In [8]:
df1 = pd.DataFrame()
for tok in tokens:
    df_tok = df[df['Token']==tok]
    df_tok = df_tok.drop(['Token', 'Date'], axis=1)

    col_names = []
    for col in df_tok.columns:
        if col == 'Timestamp':
            col_names.append(f'{col}')
        else:
            col_names.append(f'{tok}_{col}')
        
    df_tok.columns = col_names
    #df_tok = df_tok.set_index('Timestamp', drop=True)
    
    if df1.empty:
        df1 = df_tok
    else:
        df1 = pd.merge(df1, df_tok, on='Timestamp')

In [9]:
df1.sort_values('Timestamp', inplace=True)
df1["Date"] = pd.to_datetime(df1["Timestamp"], unit='s', origin='unix')

In [10]:
df1.head()

,Timestamp,DAI_Borrowing Rate,DAI_Deposit Rate,DAI_Borrow Volume,DAI_Supply Volume,USDC_Borrowing Rate,USDC_Deposit Rate,USDC_Borrow Volume,USDC_Supply Volume,USDT_Borrowing Rate,USDT_Deposit Rate,USDT_Borrow Volume,USDT_Supply Volume,ETH_Borrowing Rate,ETH_Deposit Rate,ETH_Borrow Volume,ETH_Supply Volume,Date
0,1609471800,0.073195,0.050982,1.069964e+09,6.196481e+10,0.087046,0.066993,7.285430e+08,4.063042e+10,0.099588,0.077548,6.430536e+07,3.696225e+09,0.022952,0.000489,30553.654354,5.663257e+07,2021-01-01 03:30:00
1,1609473600,0.073101,0.050912,1.069961e+09,6.197050e+10,0.087053,0.066998,7.285469e+08,4.063019e+10,0.094890,0.073569,6.407877e+07,3.700299e+09,0.022952,0.000489,30553.703955,5.663353e+07,2021-01-01 04:00:00
2,1609475400,0.073061,0.050882,1.069972e+09,6.197354e+10,0.087058,0.067003,7.285528e+08,4.063019e+10,0.085767,0.065933,6.399418e+07,3.729213e+09,0.022951,0.000489,30553.830472,5.664534e+07,2021-01-01 04:30:00
3,1609477200,0.073436,0.051161,1.070496e+09,6.197966e+10,0.086921,0.066889,7.285713e+08,4.063660e+10,0.072946,0.055500,6.317162e+07,3.730028e+09,0.022952,0.000489,30553.786270,5.663860e+07,2021-01-01 05:00:00
4,1609479000,0.067829,0.047015,1.070566e+09,6.234580e+10,0.086312,0.066383,7.285755e+08,4.066162e+10,0.057764,0.043239,6.256095e+07,3.752927e+09,0.022952,0.000489,30553.841412,5.663440e+07,2021-01-01 05:30:00


# Utility Functions

In [11]:
#classes
# 0 - to stay within
# 1 - to drop more than or equal to 
# 2 - to increase more than or equal to 

def set_target(v, threshold):
    if v > threshold:
        return 2
    elif (v <= threshold) and (v >= -threshold):
        return 0
    elif v < - threshold:
        return 1

def get_tabpandas_multi(df, token, threshold, target_window, n_timepoint_inp):

    df = df.reset_index(drop=True)
    feature_cols = ['DAI_Borrowing Rate', 'DAI_Deposit Rate', 'DAI_Borrow Volume', 'DAI_Supply Volume', 
                    'USDC_Borrowing Rate', 'USDC_Deposit Rate', 'USDC_Borrow Volume', 'USDC_Supply Volume', 
                    'USDT_Borrowing Rate', 'USDT_Deposit Rate', 'USDT_Borrow Volume', 'USDT_Supply Volume',
                    'ETH_Borrowing Rate', 'ETH_Deposit Rate', 'ETH_Borrow Volume', 'ETH_Supply Volume']

    target_column = f'{token}_Borrowing Rate'
    target = f'{token}_Target'

    cols_names = []
    for j in range(n_timepoint_inp):
        for col in feature_cols:
            cols_names.append(f'{col}_t-{n_timepoint_inp -j-1}')
    cols_names += [target]

    pairs = []
    for i, row in tqdm(df.iterrows()):
        if i < (len(df)-target_window-n_timepoint_inp-1):
            features = df.loc[i:i+n_timepoint_inp-1, feature_cols].values
            features = [item for sublist in features for item in sublist]

            val =  ((df.loc[i+n_timepoint_inp-1+target_window, target_column]) / (df.loc[i+n_timepoint_inp-1, target_column]) - 1)
            targs = set_target(val, threshold)

            features += [targs]
            pairs.append(features)

    df = pd.DataFrame(pairs, columns=cols_names).dropna().reset_index(drop=True)
    
    n_min = df[target].value_counts()[2]

    df_a = df[df[target] == 0].sample(n=n_min)
    df_b = df[df[target] != 0]
    df = pd.concat([df_a, df_b]).reset_index(drop=True)
    
    splits = RandomSplitter(seed=101)(range_of(df))

    cont_names = list(df.columns[:-1])
    y_names = target
    procs = [Categorify, FillMissing, Normalize]
    y_block = CategoryBlock()

    to = TabularPandas(df, procs=procs, cont_names=cont_names, y_names=y_names, y_block=y_block, splits=splits)
    dls = to.dataloaders(bs=128)

    return df, to, dls

@patch
def plot_confusion_matrix(self:ClassificationInterpretation, normalize=False, title='Confusion matrix', cmap="Blues", norm_dec=2,
                          plot_txt=True, **kwargs):
    "Plot the confusion matrix, with `title` and using `cmap`."
    # This function is mainly copied from the sklearn docs
    cm = self.confusion_matrix()
    if normalize: cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig = plt.figure(**kwargs)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(self.vocab))
    plt.xticks(tick_marks, self.vocab, rotation=90)
    plt.yticks(tick_marks, self.vocab, rotation=0)

    if plot_txt:
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            coeff = f'{cm[i, j]:.{norm_dec}f}' if normalize else f'{cm[i, j]}'
            plt.text(j, i, coeff, horizontalalignment="center", verticalalignment="center", color="white" if cm[i, j] > thresh else "black")

    ax = fig.gca()
    ax.set_ylim(len(self.vocab)-.5,-.5)

    plt.tight_layout()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.grid(False)

    return fig


def get_tab_learner_train(model_type, token, n_timepoint, target_window, threshold):
    
    df, to, dls = get_tabpandas_multi(df1, token, threshold, target_window, n_timepoint)
    sm = SaveModelCallback(fname=f'{path}/models/{model_type}_{token}_{n_timepoint}_{target_window}')

    learn = tabular_learner(dls, 
                        [200,100], 
                        metrics=accuracy,
                        n_out=3,
                        cbs=sm)
    
    learn.fit_one_cycle(100, 0.03)
    
    get_preds_save_classification(learn, token, threshold, target_window, n_timepoint)
    
    return learn

class LSTMDataset(Dataset):
    def __init__(self, df, 
                 n_timepoint_input, 
                 numfeatpertimepoint):
        self.df = df.items.reset_index(drop=True)
        self.n_sequence = n_timepoint_input
        self.numfeatures = numfeatpertimepoint
        self.vocab = ([0, 1, 2], )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.loc[index,:]
        target = tensor(row.iloc[-1]).long()
        features = [tensor(list(row[i*self.numfeatures:(i*self.numfeatures+self.numfeatures)].values)) for i in range(self.n_sequence)]
        features = torch.stack(features)
        return (features, target)


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2, num_classes=3):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :]
        out = self.fc(out)

        return out


def get_lstm_learner_train(model_type, token, n_timepoint, target_window, threshold):
    
    df, to, dls = get_tabpandas_multi(df1, token, threshold, target_window, n_timepoint)
    train_dset = LSTMDataset(to.train, n_timepoint, 16)
    valid_dset = LSTMDataset(to.valid, n_timepoint, 16)

    dls2 = DataLoaders.from_dsets(train_dset, valid_dset, bs=128)
    
    hidden_size = 128
    
    sm = SaveModelCallback(fname=f'{path}/models/{model_type}_{token}_{n_timepoint}_{target_window}')
    model = LSTMModel(16, hidden_size)
    learn = Learner(dls2, 
                model, 
                metrics=accuracy, 
                loss_func=CrossEntropyLossFlat(),
                cbs=sm)
    learn.fit_one_cycle(50, 0.03)    
    get_preds_save_classification(learn, model_type, token, n_timepoint, target_window, threshold)
    
    return df, dls, dls2, learn


def get_preds_save_classification(learn, model_type, token, n_timepoint, target_window, threshold):
    preds, targs = learn.get_preds(dl=learn.dls.valid)
    targs = targs.squeeze()
    preds = torch.argmax(preds, 1)
    acc = sum(preds==targs)/len(preds) * 100
    interp = ClassificationInterpretation.from_learner(learn)
    p = interp.plot_confusion_matrix(title=f'{model_type} {token} n_timepoint {n_timepoint} target_window {target_window} accuracy {acc}')
    p.savefig(f'{path}/results/{model_type} {token} n_timepoint {n_timepoint} target_window {target_window} threshold {threshold}.png')

## TabModel Classification

In [12]:
#target_window = 5
#n_timepoint = 1
#threshold = 0.05
#token = 'DAI'
#model_type = 'TabClassification'

In [13]:
#learn = get_tab_learner_train(model_type, token, n_timepoint, target_window, threshold)

## LSTM

In [14]:
threshold = 0.05
model_type = 'LSTMClassi'

In [ ]:
for tok in ['USDT', 'ETH']:
   for ntp in [1, 3, 10, 20, 48]:
       for tw in [5, 10, 20, 48]:
           df, dls, dls2, learn = get_lstm_learner_train(model_type, tok, ntp, tw, threshold)

13766it [00:11, 1164.02it/s]


epoch,train_loss,valid_loss,accuracy,time
0,1.034635,0.963402,0.492537,00:02
1,0.970600,0.883336,0.579957,00:02
2,0.938182,0.891487,0.596304,00:03
3,0.919047,0.884026,0.576404,00:04
4,0.912336,0.870924,0.607676,00:02
5,0.909745,0.868033,0.591329,00:02
6,0.908735,0.869050,0.597726,00:02
7,0.908723,0.864383,0.609097,00:02
8,0.899906,0.856173,0.601279,00:02
9,0.893393,0.839247,0.627576,00:02


Better model found at epoch 0 with valid_loss value: 0.9634019732475281.
Better model found at epoch 1 with valid_loss value: 0.8833363652229309.
Better model found at epoch 4 with valid_loss value: 0.8709235191345215.
Better model found at epoch 5 with valid_loss value: 0.8680328726768494.
Better model found at epoch 7 with valid_loss value: 0.8643825054168701.
Better model found at epoch 8 with valid_loss value: 0.8561733961105347.
Better model found at epoch 9 with valid_loss value: 0.8392470479011536.
Better model found at epoch 14 with valid_loss value: 0.8329470157623291.
Better model found at epoch 16 with valid_loss value: 0.8292737007141113.
Better model found at epoch 17 with valid_loss value: 0.8141411542892456.
Better model found at epoch 18 with valid_loss value: 0.8075664043426514.
Better model found at epoch 23 with valid_loss value: 0.7877395749092102.
Better model found at epoch 27 with valid_loss value: 0.7855439186096191.
Better model found at epoch 28 with valid_los

13766it [00:11, 1148.17it/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.998447,0.922857,0.552215,00:03
1,0.909214,0.854012,0.628165,00:03
2,0.879519,0.845969,0.625527,00:03
3,0.864883,0.839491,0.632911,00:03
4,0.863895,0.843966,0.610232,00:03
5,0.860397,0.853681,0.598101,00:03
6,0.854815,0.841295,0.627637,00:03
7,0.841955,0.829599,0.623418,00:03
8,0.841228,0.845821,0.621308,00:03
9,0.827907,0.808067,0.641350,00:03


...

epoch,train_loss,valid_loss,accuracy,time
0,0.928002,0.883477,0.591534,00:47
1,0.882494,0.852288,0.626455,00:47
2,0.869651,0.856992,0.620106,00:47
3,0.857583,0.843648,0.634392,00:47
4,0.853596,0.853737,0.624868,00:47
5,0.845283,0.848450,0.616402,00:47
6,0.844910,0.830177,0.637037,00:47
7,0.842151,0.844088,0.630688,00:46
8,0.854265,0.846142,0.628042,00:47
9,0.860274,0.867685,0.603704,00:47


Better model found at epoch 0 with valid_loss value: 0.883476972579956.
Better model found at epoch 1 with valid_loss value: 0.8522884249687195.
Better model found at epoch 3 with valid_loss value: 0.8436477184295654.
Better model found at epoch 6 with valid_loss value: 0.8301773071289062.
Better model found at epoch 11 with valid_loss value: 0.8178257942199707.
Better model found at epoch 15 with valid_loss value: 0.8004711270332336.
Better model found at epoch 16 with valid_loss value: 0.7848916053771973.
Better model found at epoch 24 with valid_loss value: 0.7742773294448853.
Better model found at epoch 26 with valid_loss value: 0.7499403357505798.
Better model found at epoch 27 with valid_loss value: 0.7392308712005615.
Better model found at epoch 29 with valid_loss value: 0.7380632162094116.
Better model found at epoch 30 with valid_loss value: 0.7092679738998413.
Better model found at epoch 31 with valid_loss value: 0.6777198314666748.
Better model found at epoch 32 with valid_l

13766it [00:15, 908.16it/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.879733,0.830884,0.634858,00:56
1,0.831720,0.795576,0.653159,00:56


Better model found at epoch 0 with valid_loss value: 0.8308836221694946.
Better model found at epoch 1 with valid_loss value: 0.7955760359764099.
